# 1D wave equation

### PDE and boundary conditions
The homogeneous wave equation reads

 $$\frac{\partial^2 c}{\partial t^2} = u^2 \frac{\partial^2 c}{\partial x^2},$$

 where $c$ is the independent variable (displacement, concentration, temperature, etc)
 , $u$ is the characteristic velocity.
 
 The inhomogeneous wave equation with initial conditions on the value and derivative of the concentration takes the form 
 
 \begin{align}
     & \frac{\partial^2 c\left(x,t\right)}{\partial t^2} - u^2 \frac{\partial^2 c\left(x,t\right)}{\partial x^2} = s\left(x,t\right) \\
     & c\left(x, 0\right) = f\left(x\right) \\
     & \frac{\partial c}{\partial t}\left(x, 0\right) = g\left(x\right)
 \end{align}
 
 Written by Ali A. Eftekhari
 Last checked: June 2021
 
 Ported to python by Gavin M. Weir June, 2023


In [ ]:
# Enable interactive plotting
%matplotlib notebook   

In [ ]:
import pyfvtool as pyfvm
from pyfvtool import *
import numpy as np

# for animation and visualization
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [ ]:
# add a utility function to extract 1D profile
# this may later become a method of CellVariable
def get_CellVariable_profile1D(cv):
    x = np.hstack([cv.domain.facecenters.x[0],
                   cv.domain.cellcenters.x,
                   cv.domain.facecenters.x[-1]])
    phi = np.hstack([0.5*(cv.value[0]+cv.value[1]),
                     cv.value[1:-1],
                     0.5*(cv.value[-2]+cv.value[-1])])
    # The size of the ghost cell is always equal to the size of the 
    # first (or last) cell within the domain. The value at the
    # boundary can therefore be obtained by direct averaging with a
    # weight factor of 0.5.
    return (x, phi)

In [ ]:
# c=@(x)(x.^2);
# dc=@(x)(2*x);

def c(x):
    return x*x

def dc(x):
    return 2.0*x


Lx = float(1.0)   # length of domain
Nx = int(200)     # number of cells in domain
dx = Lx/Nx        # step-size in uniform 1d grid
m = createMesh1D(Nx, Lx) # 1D domain

x_face=m.facecenters.x   # face positions
x_cell=m.cellcenters.x   # node positions

# Boundary conditions
BC = createBC(m)
BC.left.periodic = True
BC.right.periodic = True

# initial value on grid
u0 = np.abs(np.sin(x_cell/Lx*10*np.pi))

# Velocity on grid nodes (cell centers)
u_old = createCellVariable(m, u0);
u_val = u_old

dt = 0.1  # time-step for calculations

# Initialize the face concenrtation values and their derivatives at the faces 
c_face = createFaceVariable(m, 0.0)     # concentration values
c_face.xvalue = c(x_face)                         
dc_cell = createCellVariable(m, dc(x_cell))  # concentration derivative

# Convection and inhomogeneity act as sources
Mconv = convectionUpwindTerm(c_face)
Ms = linearSourceTerm(dc_cell)            # 

Mbc, RHSbc = pyfvm.boundary.boundaryCondition1D(BC)

# ========= #

ui = []
for ii in range(2000): # 1:1000
    Mt, RHSt = transientTerm(u_old, dt, 1.0)
    
    # Coefficient matrix
    M = Mt+Mconv-Ms+Mbc   # transient term + convection - linear source + value at boundary
    
    # Define right-hand side of the equation
    RHS = RHSt+RHSbc   # right-hand side is the time-derivative + boundary condition (source terms)
    
    # Solve the PDE 
    ui.append( solvePDE(m, M, RHS) )
    
    # Downgrade the current solution to a previous time-step
    u_old = ui[ii]
    # visualizeCells(ui[ii])


In [ ]:
# Plotting and visualization of the 1D wave equation solution  

hfig1, ax1 = plt.subplots()

xx, uu = pyfvm.utilities.get_CellVariable_profile1D(ui[0])
ax1.plot(xx, uu, 'b-', label='initial value')

xx, uu = pyfvm.utilities.get_CellVariable_profile1D(ui[-1])
ax1.plot(xx, uu, 'r-', label='final value')

ax1.set_xlim((0, Lx))
ax1.set_ylim((0.0, 1.0))

ax1.set_xlabel('x')
ax1.set_ylabel('c')
ax1.set_title('1D wave equation solution')
ax1.legend(fontsize=8)

# ========= #

# create a figure to handle the solution animation
#   pre-generate the handles to the axes and the line
hfig2 = plt.figure()
ax2 = plt.axes(xlim=(0, Lx), ylim=(0.0, 1.0))
line, = ax2.plot([], [], 'k-', lw=3, label='Numerical solution')

ax2.set_xlabel('x')
ax2.set_ylabel('c')
ax2.set_title('1D wave equation solution')
ax2.legend(fontsize=8)


def init():
    line.set_data([], [])
    return line, 

def animate(ii):
    # x, y = get_CellVariable_profile1D(ui[ii])

    global ui
    line.set_data(*pyfvm.utilities.get_CellVariable_profile1D(ui[ii]))
    return line, 

anim = FuncAnimation(hfig2, animate, init_func=init,
            frames=400, interval=20, blit=True)